# MOVIES RECOMMENDER SYSTEM

In [1]:
import numpy as np
import pandas as pd
import ast
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

In [2]:
#creating data frame for movies and credits
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
# merging based on title
movies= movies.merge(credits,on='title')

In [6]:
# will keep movie id, title, genres, overview, keywords, cast, crew

In [7]:
movies = movies[['movie_id','title','genres', 'overview', 'keywords', 'cast', 'crew' ]]

In [8]:
movies.duplicated().sum()

0

In [9]:
movies.isnull().sum()

movie_id    0
title       0
genres      0
overview    3
keywords    0
cast        0
crew        0
dtype: int64

In [10]:
movies.dropna(inplace=True) # drop these three null values

In [11]:
movies.head()

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [12]:
# looking into genres, which value to take. Need the values of key "name" like, Adventure, Action, Fantasy, Science Fiction.
movies.iloc[10].genres

'[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}, {"id": 878, "name": "Science Fiction"}]'

In [13]:
# creating funtions to fetch the important data


# calling ast library to convert list of strings into valid list.
# eg, '[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}, {"id": 878, "name": "Science Fiction"}]'
# converted into [Adventure, Action, Fantasy, Science Fiction]
def convert(obj):
    L = []
    obj = ast.literal_eval(obj)
    for i in obj:
        L.append(i['name'])
    return L


# getting info from cast section, where  first four dictionary have lead cast names, rest are side actors
def convert4(obj):
    L = []
    n = 0
    obj = ast.literal_eval(obj)
    for i in obj:
        if n != 4:
            L.append(i['name'])
            n+=1
        else:
            break
    return L



# Getting the info of director as many love the movies of same kind of directors as well
def fetch_director(obj):
    L = []
    obj = ast.literal_eval(obj)
    for i in obj:
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [14]:
movies['genres'] = movies['genres'].apply(convert) # calling convert function to get genres
movies['keywords'] = movies['keywords'].apply(convert) # calling convert function to get keywords

In [15]:
# Calling convert4 function to get the name of top four cast of the movie
movies['cast'] = movies['cast'].apply(convert4)

In [16]:
# calling fetch_director function to get the name of director
movies['crew'] = movies['crew'].apply(fetch_director)

In [17]:
# converting each word into list using simple lambda function in overview columm
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [18]:
""" Removing space in each words in genres, keywords, cast and crew, so as to make it a unique entity for example,
sam wilson and sam edwards two different person, but later on sam will be removed considering as duplicate even
though it is not. So, this function will make it like, samwilson and samedward which are two unique entities."""

movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

In [19]:
# joining other tags and making it into one column name "tags"
movies['tags'] = movies ['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

# making it into string based on space in between entities from tags column as well.
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

# converting all characters into lower cases.
movies['tags'] = movies['tags'].apply(lambda x: x.lower())

In [20]:
# creating new data frame with, id, title and tags only.
finalDF = movies [['movie_id', 'title', 'tags']]

In [21]:
# Final Data Frame on which model implementing can be done.
finalDF.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


# ____

# Now onto getting value info from the data frame

1. Now, the headache is how to compare other movies and find simliar words.
2. Answer to this is, convert these into numbers, because ML understand the numbers the best.
3. Now, to convert these texts, we can convert it into vector values
4. Thus, we can now compare these movies based on vactor values by various methods like minimum distance or minimum angle between them.

In [22]:
"""Converting text into vectors using simple method called "Bag of words", with most occuring text on top and getting
upto 5000 values and removing the normal verbal words called 'stop word' of English language"""

cv = CountVectorizer(max_features = 5000, stop_words = 'english')
vectors = cv.fit_transform(finalDF['tags']).toarray() # applying it on data frame

In [23]:
# first movie avatar in the form of vector
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [31]:
# getting the most common 5000 unique values created by count vectorizer
featureNames = cv.get_feature_names()

In [ ]:
# Applying stemming which makes words like action, actions, actioned into action only, thus providing more efficiency
# using PorterStemmer class from nltk library
ps = PorterStemmer()

# function to do stemming
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
# eg for stemming
ps.stem('actions', 'actioned')

In [ ]:
finalDF['tags'] = finalDF['tags'].apply(stem) # applying stem function

In [ ]:
finalDF['tags'][0] # This is how tags looks finally

In [ ]:
finalDF.head()

In [ ]:
finalDF.shape

# ____

# ML Modelling

* using Cosine similarity ahead of Euclidean similarity,
* reason being, Euclidean distance fails when their is large amount of data.

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity

In [ ]:
# This Shows that similarity between 0 to 0 , 0 to 1, 0 to 2, and so on for 2D array.
# Here it shows that, 0 to 0, similarity value is 1.00, which is accurate as example avatar to avatar the similarity is maximum
pd.DataFrame(similarity).head()

# ____

# Let's make recommender function

In [ ]:
def recommend(movie):
    movie_index = finalDF[finalDF['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x: x[1])[1:6]
    """here enumerate helps to create tuple where, each movie distance with each movies is recorded with lamda function informing to sort based
    similarity based not on index basis"""
    
    for i in movies_list:
        print (finalDF.iloc[i[0]].title)

In [ ]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x: x[1])
# this shows avatar similarity and shows movie at 539 is most similar to Avatar

In [ ]:
# So here we are done, let's recommend a movie

recommend('The Avengers')

# ____

# Exporting data using Pickle

In [ ]:
pickle.dump(finalDF.to_dict(),open("moviesInDictionary.pkl","wb"))

In [ ]:
pickle.dump(similarity,open("similarity.pkl","wb"))